![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.

## Funkcje okienne w Spark

Funkcje okienne (ang. window functions) w Spark zostały dodane w wersji 1.4, ale nadal są mniej znaną funkcją systemu; zobacz [wpis na blogu Databrics](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html) przybliżający te funkce. Ninejszy notebook przejdzie przez najważniejsze funkcje okienne dostępne w Spark.

## Pobieranie danych

Zestaw danych do tego ćwiczenia to dane udostępnione przez IBM i są to przykładowe dane sprzezażowe dostępne [tutaj](https://www.ibm.com/communities/analytics/watson-analytics-blog/sales-products-sample-data/). Poniższy kod pobierze dane automatycznie.

In [1]:
import os
import urllib.request

if not os.path.exists('data'):
    os.mkdir('data')

url = 'https://community.watsonanalytics.com/wp-content/uploads/2015/08/WA_Sales_Products_2012-14.csv'    
csv_file = 'data/WA_Sales_Products_2012-14.csv'

urllib.request.urlretrieve(url, 'data/WA_Sales_Products_2012-14.csv');

Inicjujemy sesje Spark jak zwykle.

In [2]:
import pyspark
import pyspark.sql.functions as func

spark = pyspark.sql.SparkSession.builder\
    .appName('why_sql')\
    .getOrCreate()

In [6]:
spark.sparkContext

<SparkContext master=local[*] appName=why_sql>

Dane są w formacie CSV, więc możemy użyć wbudowany czytnik CSV; dane są dość małe i dobrej jakości, więc możemy pozwolić sobie na automatyczne odkrycie schematu.

In [3]:
sales = spark.read.csv(csv_file, header=True, inferSchema=True)

# This is needed for string queries, you don't need to do it using DF syntax
sales.createTempView('sales') 

sales.printSchema()
sales.show()

root
 |-- Retailer country: string (nullable = true)
 |-- Order method type: string (nullable = true)
 |-- Retailer type: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Product type: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Gross margin: double (nullable = true)

+----------------+-----------------+-------------+--------------------+--------------------+--------------------+----+-------+---------+--------+------------+
|Retailer country|Order method type|Retailer type|        Product line|        Product type|             Product|Year|Quarter|  Revenue|Quantity|Gross margin|
+----------------+-----------------+-------------+--------------------+--------------------+--------------------+----+-------+---------+--------+------------+
|   United States|              Fax|Outdoors Sho

## Czym są funkcje okienne?

Spróbujmu znaleźć produkty o największym dochodzie (Reveniue)

In [4]:
sales.select('Product', 'Product line', 'Product type', 'Year', 'Revenue')\
    .orderBy(func.desc('Revenue'))\
    .limit(10)\
    .show()

+--------------------+--------------------+-------------+----+----------+
|             Product|        Product line| Product type|Year|   Revenue|
+--------------------+--------------------+-------------+----+----------+
|Hailstorm Titaniu...|      Golf Equipment|        Woods|2014|1635687.96|
|        Star Gazer 2|   Camping Equipment|        Tents|2014| 1486717.1|
|           Star Lite|   Camping Equipment|        Tents|2013|1415141.91|
|Hailstorm Titaniu...|      Golf Equipment|        Woods|2014| 1388659.5|
|        Star Gazer 2|   Camping Equipment|        Tents|2013| 1335112.9|
|        Star Gazer 2|   Camping Equipment|        Tents|2013| 1311874.3|
|                Zone|Personal Accessories|      Eyewear|2013|1230450.95|
|Hailstorm Titaniu...|      Golf Equipment|        Woods|2013|1226669.53|
|           Star Lite|   Camping Equipment|        Tents|2012|1210413.68|
|  Hibernator Extreme|   Camping Equipment|Sleeping Bags|2013|1199043.84|
+--------------------+----------------

Ale danych jest więcej w zależności np od lini, typu lub roku.

### Zadanie

1. Znajdź ile jest i jakie są unikalne lini (Product line), typy (Product type) produkcji i lata raportu (Year)?
1. Pogrupuj po tych elementach i zobacz jakie są zakresy dochodu.

Jako, że zmiennych opisujących, które wpływają na dochód jest sporo, takie globalne sortowanie jest średnio informacyjne. Bardziej użyteczne jest np. grupowanie po latach. Jak zrobić zatem takie grupowanie aby dostać 3 najlepiej zarabiające produkty w danym roku? Możemy użyć funkcji okiennej `rank`.

In [8]:
sales.select('Product line').distinct().show(truncate=False)

+------------------------+
|Product line            |
+------------------------+
|Camping Equipment       |
|Golf Equipment          |
|Mountaineering Equipment|
|Outdoor Protection      |
|Personal Accessories    |
+------------------------+



In [17]:
sales.select('Product type').distinct().show()

+--------------------+
|        Product type|
+--------------------+
|             Eyewear|
|          Navigation|
|Climbing Accessories|
|               Tents|
|             Putters|
|               Tools|
|               Woods|
|   Insect Repellents|
|           First Aid|
|               Packs|
|          Binoculars|
|            Lanterns|
|        Cooking Gear|
|              Safety|
|             Watches|
|                Rope|
|               Irons|
|       Sleeping Bags|
|           Sunscreen|
|              Knives|
+--------------------+
only showing top 20 rows



In [18]:
sales.select('Year').distinct().show()

+----+
|Year|
+----+
|2013|
|2014|
|2012|
+----+



In [21]:
sales.describe().toPandas()

,summary,Retailer country,Order method type,Retailer type,Product line,Product type,Product,Year,Quarter,Revenue,Quantity,Gross margin
0,count,88475,88475,88475,88475,88475,88475,88475,88475,88475,88475,87894
1,mean,None,None,None,None,None,Infinity,2012.855281152868,None,42638.292909070675,780.5861655834982,0.44971769825050273
2,stddev,None,None,None,None,None,NaN,0.7783422272719562,None,65784.02411117143,1541.6454222228506,0.12364159169974814
3,min,Australia,E-mail,Department Store,Camping Equipment,Binoculars,Aloe Relief,2012,Q1 2012,0.0,1,-12.85367817
4,max,United States,Web,Warehouse Store,Personal Accessories,Woods,Zone,2014,Q4 2013,1635687.96,67875,0.77047619


In [22]:
sales.groupby('year')\
    .agg(func.count('Product line'), func.max('Revenue'), func.min('Revenue'), func.mean('Revenue'))\
    .show()

+----+-------------------+------------+------------+------------------+
|year|count(Product line)|max(Revenue)|min(Revenue)|      avg(Revenue)|
+----+-------------------+------------+------------+------------------+
|2013|              33023|  1415141.91|         0.0|45298.461705477916|
|2014|              21324|  1635687.96|         0.0| 52398.06199915586|
|2012|              34128|  1210413.68|         0.0|33966.115511017335|
+----+-------------------+------------+------------+------------------+



In [23]:
sales\
    .agg(func.count('Product line'), func.max('Revenue'), func.min('Revenue'), func.mean('Revenue'))\
    .show()

+-------------------+------------+------------+------------------+
|count(Product line)|max(Revenue)|min(Revenue)|      avg(Revenue)|
+-------------------+------------+------------+------------------+
|              88475|  1635687.96|         0.0|42638.292909070675|
+-------------------+------------+------------+------------------+



In [24]:
w = pyspark.sql.Window\
    .partitionBy('Year')\
    .orderBy(func.desc('Revenue'))

year_with_rank = sales.withColumn('rank', func.rank().over(w))\
    .select('Product', 'Product line', 'Product type', 'Year', 'Revenue', 'Rank')

year_with_rank.show()

+--------------------+--------------------+-------------+----+----------+----+
|             Product|        Product line| Product type|Year|   Revenue|Rank|
+--------------------+--------------------+-------------+----+----------+----+
|           Star Lite|   Camping Equipment|        Tents|2013|1415141.91|   1|
|        Star Gazer 2|   Camping Equipment|        Tents|2013| 1335112.9|   2|
|        Star Gazer 2|   Camping Equipment|        Tents|2013| 1311874.3|   3|
|                Zone|Personal Accessories|      Eyewear|2013|1230450.95|   4|
|Hailstorm Titaniu...|      Golf Equipment|        Woods|2013|1226669.53|   5|
|  Hibernator Extreme|   Camping Equipment|Sleeping Bags|2013|1199043.84|   6|
|                Zone|Personal Accessories|      Eyewear|2013| 1118965.3|   7|
|           Star Lite|   Camping Equipment|        Tents|2013|1052750.28|   8|
|            Infinity|Personal Accessories|      Watches|2013|  973804.4|   9|
|        Star Gazer 2|   Camping Equipment|        T

In [28]:
year_with_rank.where('Rank <= 3').select('Product', 'Year').show(truncate=False)

+----------------------------+----+
|Product                     |Year|
+----------------------------+----+
|Star Lite                   |2013|
|Star Gazer 2                |2013|
|Star Gazer 2                |2013|
|Hailstorm Titanium Woods Set|2014|
|Star Gazer 2                |2014|
|Hailstorm Titanium Woods Set|2014|
|Star Lite                   |2012|
|Zone                        |2012|
|Zone                        |2012|
+----------------------------+----+



## Zadanie

1. Wykonaj tę samą funkcję `rank` ale dla innej columny.
1. Wykonaj funkcję `rank` po kilku kolumnach.
1. Usuń kolumnę `rank`.
1. ★ Ile więcej zarobiły te produkty od średniej w danym roku?

Można to samo zapisać w formie SQL jak poniżej. Funkcja `RANK` jest zreszą częścią standardu ANSI SQL 1999.

In [41]:
w = pyspark.sql.Window\
    .partitionBy('Year', 'Quarter')\
    .orderBy(func.desc('Revenue'))

year_with_rank = sales.withColumn('rank', func.rank().over(w))\
    .select('Product', 'Product line', 'Product type', 'Year', 'Quarter', 'Revenue', 'Rank')

year_with_rank.where('Rank <= 3').drop('Rank').orderBy('Year', 'Quarter').show()

+--------------------+--------------------+------------+----+-------+----------+
|             Product|        Product line|Product type|Year|Quarter|   Revenue|
+--------------------+--------------------+------------+----+-------+----------+
|           Star Lite|   Camping Equipment|       Tents|2012|Q1 2012|1210413.68|
|        Star Gazer 2|   Camping Equipment|       Tents|2012|Q1 2012| 603311.94|
|                Zone|Personal Accessories|     Eyewear|2012|Q1 2012|  638000.5|
|                Zone|Personal Accessories|     Eyewear|2012|Q2 2012|  711955.2|
|           Star Lite|   Camping Equipment|       Tents|2012|Q2 2012| 706259.02|
|                Zone|Personal Accessories|     Eyewear|2012|Q2 2012|  655149.8|
|                Zone|Personal Accessories|     Eyewear|2012|Q3 2012| 1009957.9|
|           Star Lite|   Camping Equipment|       Tents|2012|Q3 2012|  654089.6|
|                Zone|Personal Accessories|     Eyewear|2012|Q3 2012|  745868.9|
|        Star Gazer 2|   Cam

In [32]:
query = """
SELECT 
    Product, 
    `Product line`,
    `Product type`,
    Year,
    Revenue,
    RANK() OVER(PARTITION BY Year ORDER BY Revenue DESC) AS Rank
FROM sales
HAVING Rank <= 3
"""
spark.sql(query).show()

+--------------------+--------------------+------------+----+----------+----+
|             Product|        Product line|Product type|Year|   Revenue|Rank|
+--------------------+--------------------+------------+----+----------+----+
|           Star Lite|   Camping Equipment|       Tents|2013|1415141.91|   1|
|        Star Gazer 2|   Camping Equipment|       Tents|2013| 1335112.9|   2|
|        Star Gazer 2|   Camping Equipment|       Tents|2013| 1311874.3|   3|
|Hailstorm Titaniu...|      Golf Equipment|       Woods|2014|1635687.96|   1|
|        Star Gazer 2|   Camping Equipment|       Tents|2014| 1486717.1|   2|
|Hailstorm Titaniu...|      Golf Equipment|       Woods|2014| 1388659.5|   3|
|           Star Lite|   Camping Equipment|       Tents|2012|1210413.68|   1|
|                Zone|Personal Accessories|     Eyewear|2012| 1042285.0|   2|
|                Zone|Personal Accessories|     Eyewear|2012| 1009957.9|   3|
+--------------------+--------------------+------------+----+---

In [43]:
w = pyspark.sql.Window.partitionBy('Year')

sales.withColumn('Avg Rev by Year', func.avg('Revenue').over(w))\
    .select('Year', 'Avg Rev by Year', 'Revenue', (func.col('Revenue') - func.col('Avg Rev by Year')).alias('Diff'))\
    .show()

+----+-----------------+---------+-------------------+
|Year|  Avg Rev by Year|  Revenue|               Diff|
+----+-----------------+---------+-------------------+
|2013|45298.46170547772| 19418.52| -25879.94170547772|
|2013|45298.46170547772| 42304.32|-2994.1417054777194|
|2013|45298.46170547772| 52266.32|  6967.858294522281|
|2013|45298.46170547772|  5211.64| -40086.82170547772|
|2013|45298.46170547772| 51714.46|   6415.99829452228|
|2013|45298.46170547772|112037.76|  66739.29829452228|
|2013|45298.46170547772| 22786.56|-22511.901705477718|
|2013|45298.46170547772| 12559.96| -32738.50170547772|
|2013|45298.46170547772| 13101.88| -32196.58170547772|
|2013|45298.46170547772|  3494.05|-41804.411705477716|
|2013|45298.46170547772| 12967.05| -32331.41170547772|
|2013|45298.46170547772|  32832.0|-12466.461705477719|
|2013|45298.46170547772| 57900.38| 12601.918294522278|
|2013|45298.46170547772|  43093.9|-2204.5617054777176|
|2013|45298.46170547772|  22847.5| -22450.96170547772|
|2013|4529

Podobny efekt daje funkcja `dense_rank`.

In [46]:
w = pyspark.sql.Window\
    .partitionBy('Product type')\
    .orderBy('Year')

with_dense_rank = sales.withColumn('Dense rank', func.rank().over(w))\
    .select('Product', 'Product line', 'Product type', 'Year', 'Revenue', 'Dense rank')

with_dense_rank.show()

+------------+--------------------+------------+----+--------+----------+
|     Product|        Product line|Product type|Year| Revenue|Dense rank|
+------------+--------------------+------------+----+--------+----------+
|   Polar Sun|Personal Accessories|     Eyewear|2012| 7015.34|         1|
|   Polar Ice|Personal Accessories|     Eyewear|2012|  3825.8|         1|
|       Capri|Personal Accessories|     Eyewear|2012| 10838.9|         1|
|     Cat Eye|Personal Accessories|     Eyewear|2012| 4428.85|         1|
|       Dante|Personal Accessories|     Eyewear|2012| 9759.75|         1|
|     Fairway|Personal Accessories|     Eyewear|2012| 8241.35|         1|
|     Inferno|Personal Accessories|     Eyewear|2012| 12935.0|         1|
|     Maximus|Personal Accessories|     Eyewear|2012|  9325.0|         1|
|      Trendi|Personal Accessories|     Eyewear|2012|  9104.3|         1|
|        Zone|Personal Accessories|     Eyewear|2012|  4574.3|         1|
|   Polar Sun|Personal Accessories|   

In [47]:
with_dense_rank.groupBy('year')\
    .agg(func.first('Dense rank'))\
    .orderBy('Year')\
    .show()

+----+------------------------+
|year|first(Dense rank, false)|
+----+------------------------+
|2012|                       1|
|2013|                    5629|
|2014|                   11421|
+----+------------------------+



### Zadanie

1. Zobacz co się stanie po zamianie `dense_rank` na `rank`?

Jeżeli tylko potrzebny jest numer kolejnych elementów a nie ich ranga, można użyc funcji `row_number`.

In [50]:
w = pyspark.sql.Window\
    .partitionBy('Product type')\
    .orderBy('Year')

sales.withColumn('Row', func.row_number().over(w))\
    .select('Product', 'Product line', 'Product type', 'Year', 'Revenue', 'Row')\
    .show()

+------------+--------------------+------------+----+--------+---+
|     Product|        Product line|Product type|Year| Revenue|Row|
+------------+--------------------+------------+----+--------+---+
|   Polar Sun|Personal Accessories|     Eyewear|2012| 7015.34|  1|
|   Polar Ice|Personal Accessories|     Eyewear|2012|  3825.8|  2|
|       Capri|Personal Accessories|     Eyewear|2012| 10838.9|  3|
|     Cat Eye|Personal Accessories|     Eyewear|2012| 4428.85|  4|
|       Dante|Personal Accessories|     Eyewear|2012| 9759.75|  5|
|     Fairway|Personal Accessories|     Eyewear|2012| 8241.35|  6|
|     Inferno|Personal Accessories|     Eyewear|2012| 12935.0|  7|
|     Maximus|Personal Accessories|     Eyewear|2012|  9325.0|  8|
|      Trendi|Personal Accessories|     Eyewear|2012|  9104.3|  9|
|        Zone|Personal Accessories|     Eyewear|2012|  4574.3| 10|
|   Polar Sun|Personal Accessories|     Eyewear|2012| 25787.7| 11|
|   Polar Ice|Personal Accessories|     Eyewear|2012| 20835.1|

Funkcje okienne mogą też być numeryczne. Przykładowo możemy średnią wartość przychodu dla poszczególnych lat i typu produktu.

In [52]:
w = pyspark.sql.Window\
    .partitionBy('Product type')\
    .orderBy('Revenue')

w2 = pyspark.sql.Window\
    .partitionBy('Product line')
    
sales.withColumn('Rank', func.rank().over(w))\
    .withColumn('Rev avg', func.avg('Revenue').over(w2))\
    .select('Product', 'Product line', 'Product type', 'Year', 'Revenue', 'Rev avg', 'Rank')\
    .where('Rank <= 3')\
    .show()

+--------------------+-----------------+------------+----+-------+-----------------+----+
|             Product|     Product line|Product type|Year|Revenue|          Rev avg|Rank|
+--------------------+-----------------+------------+----+-------+-----------------+----+
|            Star Peg|Camping Equipment|       Tents|2012|1334.52|50512.76144011909|   1|
|            Star Peg|Camping Equipment|       Tents|2012|1409.96|50512.76144011909|   2|
|            Star Peg|Camping Equipment|       Tents|2012|1432.18|50512.76144011909|   3|
|Canyon Mule Climb...|Camping Equipment|       Packs|2014|1998.36|50512.76144011909|   1|
|Canyon Mule Climb...|Camping Equipment|       Packs|2014|1998.36|50512.76144011909|   1|
|Canyon Mule Climb...|Camping Equipment|       Packs|2014|2152.08|50512.76144011909|   3|
|   Firefly Mapreader|Camping Equipment|    Lanterns|2014|    0.0|50512.76144011909|   1|
|   Firefly Mapreader|Camping Equipment|    Lanterns|2014|    0.0|50512.76144011909|   1|
|   Firefl

Można też definiować przesówne okna. Przykładowo możemy znajdować maksymalną wartość w przesównym oknie rozmiaru 3, z wartościami jeden przed i jeden za obecną wartością.

In [55]:
w = pyspark.sql.Window\
    .partitionBy('Product')\
    .rowsBetween(-1, 1)

sales.select('Product', 
             'Revenue', 
             func.max('Revenue').over(w).alias('Max rev window'))\
    .show()

+-------+---------+--------------+
|Product|  Revenue|Max rev window|
+-------+---------+--------------+
|Fairway|  8241.35|      36209.55|
|Fairway| 36209.55|      36209.55|
|Fairway|  11646.7|      36209.55|
|Fairway| 29237.65|       42919.5|
|Fairway|  42919.5|     130874.25|
|Fairway|130874.25|     130874.25|
|Fairway| 47251.75|     209136.85|
|Fairway|209136.85|     209136.85|
|Fairway| 11545.95|     209136.85|
|Fairway| 21903.05|      21903.05|
|Fairway|  13339.3|       60853.0|
|Fairway|  60853.0|       60853.0|
|Fairway| 27907.75|       60853.0|
|Fairway| 12916.15|      27907.75|
|Fairway|  6186.05|      15092.35|
|Fairway| 15092.35|      55795.35|
|Fairway| 55795.35|      55795.35|
|Fairway|   8019.7|      99641.75|
|Fairway| 99641.75|      99641.75|
|Fairway|   8100.3|      99641.75|
+-------+---------+--------------+
only showing top 20 rows



Można to równierz zapisać w notacji SQL.

In [56]:
query = """
SELECT
    Product,
    Revenue,
    MAX(Revenue) OVER(
        PARTITION BY Product
        ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
    ) AS `Max rev window`
FROM sales
"""
spark.sql(query).show()

+-------+---------+--------------+
|Product|  Revenue|Max rev window|
+-------+---------+--------------+
|Fairway|  8241.35|      36209.55|
|Fairway| 36209.55|      36209.55|
|Fairway|  11646.7|      36209.55|
|Fairway| 29237.65|       42919.5|
|Fairway|  42919.5|     130874.25|
|Fairway|130874.25|     130874.25|
|Fairway| 47251.75|     209136.85|
|Fairway|209136.85|     209136.85|
|Fairway| 11545.95|     209136.85|
|Fairway| 21903.05|      21903.05|
|Fairway|  13339.3|       60853.0|
|Fairway|  60853.0|       60853.0|
|Fairway| 27907.75|       60853.0|
|Fairway| 12916.15|      27907.75|
|Fairway|  6186.05|      15092.35|
|Fairway| 15092.35|      55795.35|
|Fairway| 55795.35|      55795.35|
|Fairway|   8019.7|      99641.75|
|Fairway| 99641.75|      99641.75|
|Fairway|   8100.3|      99641.75|
+-------+---------+--------------+
only showing top 20 rows



### Zadanie

1. Pokaż 5 najlepiej sprzedających się produktów dla danego typu sprzedawcy (Retailer type).
1. Policz ile mniej zarobił dany produkt od maksymalnej wartości przychodu w linii produktu w danym roku.
1. ★ Jaka jest różnica przychodu pomiędzy średnią dla danego roku, a średnią dla danych kwartałów.
1. ★ Policz średnią okienną dla okna o rozmiarze 5, w kolejności wielkości przychodu.

In [61]:
w = pyspark.sql.Window\
    .partitionBy('Retailer type')\
    .orderBy(func.desc('Revenue'), func.desc('Quantity'))

with_dense_rank = sales.withColumn('Dense rank', func.rank().over(w))\
    .select('Product', 'Product line', 'Product type', 'Year', 'Revenue', 'Quantity', 'Dense rank')

with_dense_rank.where('`Dense rank` <= 5').show()

+--------------------+--------------------+-------------+----+----------+--------+----------+
|             Product|        Product line| Product type|Year|   Revenue|Quantity|Dense rank|
+--------------------+--------------------+-------------+----+----------+--------+----------+
|        Star Gazer 2|   Camping Equipment|        Tents|2013| 1335112.9|    2413|         1|
|           Star Lite|   Camping Equipment|        Tents|2012|1210413.68|    3479|         2|
|           Star Lite|   Camping Equipment|        Tents|2013|1052750.28|    2994|         3|
|        Star Gazer 2|   Camping Equipment|        Tents|2012| 944385.75|    1725|         4|
|           Star Lite|   Camping Equipment|        Tents|2014| 850221.71|    2353|         5|
|Canyon Mule Journ...|   Camping Equipment|        Packs|2013| 726873.21|    2074|         1|
|Canyon Mule Journ...|   Camping Equipment|        Packs|2013| 645450.24|    1832|         2|
|Canyon Mule Weeke...|   Camping Equipment|        Packs|201

In [64]:
w = pyspark.sql.Window\
    .partitionBy('Year')\
    .orderBy(func.desc('Revenue'))

w2 = pyspark.sql.Window\
    .partitionBy('Year')
    
sales.select('Product', 'Product line', 'Product type', 'Year', 'Revenue')\
    .withColumn('Rank', func.rank().over(w))\
    .withColumn('Rev max', func.max('Revenue').over(w2))\
    .withColumn('Rev Diff', func.col('Revenue') - func.max('Revenue').over(w2))\
    .where('Rank <= 3')\
    .show()

+--------------------+--------------------+------------+----+----------+----+----------+-------------------+
|             Product|        Product line|Product type|Year|   Revenue|Rank|   Rev max|           Rev Diff|
+--------------------+--------------------+------------+----+----------+----+----------+-------------------+
|           Star Lite|   Camping Equipment|       Tents|2013|1415141.91|   1|1415141.91|                0.0|
|        Star Gazer 2|   Camping Equipment|       Tents|2013| 1335112.9|   2|1415141.91| -80029.01000000001|
|        Star Gazer 2|   Camping Equipment|       Tents|2013| 1311874.3|   3|1415141.91|-103267.60999999987|
|Hailstorm Titaniu...|      Golf Equipment|       Woods|2014|1635687.96|   1|1635687.96|                0.0|
|        Star Gazer 2|   Camping Equipment|       Tents|2014| 1486717.1|   2|1635687.96|-148970.85999999987|
|Hailstorm Titaniu...|      Golf Equipment|       Woods|2014| 1388659.5|   3|1635687.96|-247028.45999999996|
|           Star Li

In [68]:
query = """
SELECT
    Year,
    Quarter,
    AVG(Revenue),
    SUM(Revenue)
FROM sales
GROUP BY Year, Quarter WITH ROLLUP
ORDER BY Year, Quarter
"""
spark.sql(query).show()

+----+-------+------------------+--------------------+
|Year|Quarter|      avg(Revenue)|        sum(Revenue)|
+----+-------+------------------+--------------------+
|null|   null|42638.292909070675| 3.772422965130028E9|
|2012|   null|33966.115511017335|1.1591955901599996E9|
|2012|Q1 2012|34029.065861668736| 2.932284605299995E8|
|2012|Q2 2012|31604.267207452875| 2.781807599600002E8|
|2012|Q3 2012|32882.506662377156| 2.810796669499999E8|
|2012|Q4 2012|37582.000088224384| 3.067067027199992E8|
|2013|   null|45298.461705477916|1.4958911008999972E9|
|2013|Q1 2013| 40744.05245915211|3.4412426706999874E8|
|2013|Q2 2013|47540.272050224296|3.9187446250999886E8|
|2013|Q3 2013| 44663.12456177632|3.7811801253999835E8|
|2013|Q4 2013|48522.414689883095| 3.817743587800002E8|
|2014|   null| 52398.06199915586|1.1173362740699997E9|
|2014|Q1 2014| 56259.61680663243| 4.716243676899997E8|
|2014|Q2 2014| 58878.36901965604| 4.792699238200002E8|
|2014|Q3 2014| 34668.19049364712|1.6644198255999982E8|
+----+----

In [69]:
query = """
SELECT
    Year,
    Quarter,
    AVG(Revenue),
    SUM(Revenue)
FROM sales
GROUP BY Year, Quarter WITH CUBE
ORDER BY Year, Quarter
"""
spark.sql(query).show()

+----+-------+------------------+--------------------+
|Year|Quarter|      avg(Revenue)|        sum(Revenue)|
+----+-------+------------------+--------------------+
|null|   null|42638.292909070675| 3.772422965130028E9|
|null|Q1 2012|34029.065861668736| 2.932284605299995E8|
|null|Q1 2013| 40744.05245915211|3.4412426706999874E8|
|null|Q1 2014| 56259.61680663243| 4.716243676899997E8|
|null|Q2 2012|31604.267207452875| 2.781807599600002E8|
|null|Q2 2013|47540.272050224296|3.9187446250999886E8|
|null|Q2 2014| 58878.36901965604| 4.792699238200002E8|
|null|Q3 2012|32882.506662377156| 2.810796669499999E8|
|null|Q3 2013| 44663.12456177632|3.7811801253999835E8|
|null|Q3 2014| 34668.19049364712|1.6644198255999982E8|
|null|Q4 2012|37582.000088224384| 3.067067027199992E8|
|null|Q4 2013|48522.414689883095| 3.817743587800002E8|
|2012|   null|33966.115511017335|1.1591955901599996E9|
|2012|Q1 2012|34029.065861668736| 2.932284605299995E8|
|2012|Q2 2012|31604.267207452875| 2.781807599600002E8|
|2012|Q3 2